# NDIS network queries

#### Drop graph
Execute to have a clean start

In [42]:
%%sparql

drop graph <http://ndis-network.org>

### Load TRIPLETSTORE.ttl

In [44]:
%load -s s3://neptune-demo-s3workingbucket-12zo6uwzrgc4k/TRIPLETSTORE.ttl -f turtle --named-graph-uri=http://ndis-network.org

Button(description='Submit', style=ButtonStyle())

Output()

### Inspect top rows

In [46]:
%%sparql

# Prefix
PREFIX ndis: <http://ndis-network.org/>

# Most basic command
# Bring anything that meets hte pattern subject-predicate-object (From the NDIS-network graph)
SELECT ?s ?p ?o
FROM <http://ndis-network.org>
WHERE {
    #?s ndis:prvdr_nmbr "0000001932" .
    ?s ?p ?o
} LIMIT 100

#### 1. Extract Related Network of specific PrvdrNmbr

In [66]:
%%sparql

# Prefix
PREFIX ndis: <http://ndis-network.org/>


# Most basic command
# Bring all the neighbours (Providers and Persons) of a particular provider
SELECT ?tProvider ?r ?nEntity ?nType
FROM <http://ndis-network.org>
WHERE {
    VALUES ?tProvider { ndis:0000004100 } 
    {
      ?tProvider ?p1 ?r .
      ?r ?p2 ?nEntity ;
         a ?rType .
      ?nEntity a ?nType .
    
      FILTER ( ?nType = ndis:Person || ?nType = ndis:Provider)
    }
    UNION
    {
      ?nEntity ?p1 ?r ;
               a ?nType .
      ?r ?p2 ?tProvider ;
         a ?rType .  
    
      FILTER ( ?nType = ndis:Person || ?nType = ndis:Provider)
    }
    
}

{
  "head": {
    "vars": [
      "tProvider",
      "r",
      "nEntity",
      "nType"
    ]
  },
  "results": {
    "bindings": [
      {
        "tProvider": {
          "type": "uri",
          "value": "http://ndis-network.org/0000004100"
        },
        "r": {
          "type": "uri",
          "value": "http://ndis-network.org/ServiceFor_0000004100_part_29450"
        },
        "nEntity": {
          "type": "uri",
          "value": "http://ndis-network.org/part_29450"
        },
        "nType": {
          "type": "uri",
          "value": "http://ndis-network.org/Person"
        }
      },
      {
        "tProvider": {
          "type": "uri",
          "value": "http://ndis-network.org/0000004100"
        },
        "r": {
          "type": "uri",
          "value": "http://ndis-network.org/ServiceFor_0000004100_part_34795"
        },
        "nEntity": {
          "type": "uri",
          "value": "http://ndis-network.org/part_34795"
        },
        "nType": {
   

,tProvider,r,nEntity,nType
1,http://ndis-network.org/0000004100,http://ndis-network.org/ServiceFor_0000004100_part_29450,http://ndis-network.org/part_29450,http://ndis-network.org/Person
2,http://ndis-network.org/0000004100,http://ndis-network.org/ServiceFor_0000004100_part_34795,http://ndis-network.org/part_34795,http://ndis-network.org/Person
3,http://ndis-network.org/0000004100,http://ndis-network.org/PaymentRequestFor_0000001932_0000004100,http://ndis-network.org/0000001932,http://ndis-network.org/Provider
4,http://ndis-network.org/0000004100,http://ndis-network.org/PaymentRequestFor_0000001734_0000004100,http://ndis-network.org/0000001734,http://ndis-network.org/Provider


#### 2. Extract Payments in Related Network

In [67]:
%%sparql

# Prefix
PREFIX ndis: <http://ndis-network.org/>


# Most basic command
# Bring all the neighbours (Providers and Persons) of a particular provider
SELECT ?tProvider ?nEntity ?nType (SUM(?eCount) AS ?PaymentCount) (SUM(?pAmt) AS ?PaymentAmount)
FROM <http://ndis-network.org>
WHERE {
    VALUES ?tProvider { ndis:0000004100 } 
    {
      ?tProvider ?p1 ?r .
      ?r ?p2 ?nEntity ;
         a ?rType ;
         ndis:EdgeCount ?eCount ;
         ndis:PmtAmt ?pAmt .
         
      ?nEntity a ?nType .
      
      FILTER ( ?nType = ndis:Person || ?nType = ndis:Provider || ?rType = ndis:PaymentRequest || ?rType = ndis:ServiceFor)
    }
    UNION
    {
      ?nEntity ?p1 ?r ;
               a ?nType .
      ?r ?p2 ?tProvider ;
         a ?rType ;
         ndis:EdgeCount ?eCount ;
         ndis:PmtAmt ?pAmt .
    
      FILTER ( ?nType = ndis:Person || ?nType = ndis:Provider || ?rType = ndis:PaymentRequest || ?rType = ndis:ServiceFor)
    }   
}
GROUP BY ?tProvider ?nEntity ?nType
ORDER BY DESC (?PaymentAmount)                        

{
  "head": {
    "vars": [
      "tProvider",
      "nEntity",
      "nType",
      "PaymentCount",
      "PaymentAmount"
    ]
  },
  "results": {
    "bindings": [
      {
        "tProvider": {
          "type": "uri",
          "value": "http://ndis-network.org/0000004100"
        },
        "nEntity": {
          "type": "uri",
          "value": "http://ndis-network.org/part_34795"
        },
        "nType": {
          "type": "uri",
          "value": "http://ndis-network.org/Person"
        },
        "PaymentCount": {
          "datatype": "http://www.w3.org/2001/XMLSchema#integer",
          "type": "literal",
          "value": "55"
        },
        "PaymentAmount": {
          "datatype": "http://www.w3.org/2001/XMLSchema#float",
          "type": "literal",
          "value": "549.57733"
        }
      },
      {
        "tProvider": {
          "type": "uri",
          "value": "http://ndis-network.org/0000004100"
        },
        "nEntity": {
          "type": "u

,tProvider,nEntity,nType,PaymentCount,PaymentAmount
1,http://ndis-network.org/0000004100,http://ndis-network.org/part_34795,http://ndis-network.org/Person,55,549.57733
2,http://ndis-network.org/0000004100,http://ndis-network.org/0000001734,http://ndis-network.org/Provider,58,474.42548
3,http://ndis-network.org/0000004100,http://ndis-network.org/part_29450,http://ndis-network.org/Person,39,277.20175
4,http://ndis-network.org/0000004100,http://ndis-network.org/0000001932,http://ndis-network.org/Provider,67,214.57056


#### 3. Extract Total $ Payments in related network

In [68]:
%%sparql

# Prefix
PREFIX ndis: <http://ndis-network.org/>


# Most basic command
# Bring all the neighbours (Providers and Persons) of a particular provider
SELECT ?tProvider (SUM(?eCount) AS ?PaymentCount) (SUM(?pAmt) AS ?PaymentAmount)
FROM <http://ndis-network.org>
WHERE {
    VALUES ?tProvider { ndis:0000004100 } 
    {
      ?tProvider ?p1 ?r .
      ?r ?p2 ?nEntity ;
         a ?rType ;
         ndis:EdgeCount ?eCount ;
         ndis:PmtAmt ?pAmt .
         
      ?nEntity a ?nType .
      
      FILTER ( ?nType = ndis:Person || ?nType = ndis:Provider || ?rType = ndis:PaymentRequest || ?rType = ndis:ServiceFor)
    }
    UNION
    {
      ?nEntity ?p1 ?r ;
               a ?nType .
      ?r ?p2 ?tProvider ;
         a ?rType ;
         ndis:EdgeCount ?eCount ;
         ndis:PmtAmt ?pAmt .
    
      FILTER ( ?nType = ndis:Person || ?nType = ndis:Provider || ?rType = ndis:PaymentRequest || ?rType = ndis:ServiceFor)
    }   
}
GROUP BY ?tProvider 

{
  "head": {
    "vars": [
      "tProvider",
      "PaymentCount",
      "PaymentAmount"
    ]
  },
  "results": {
    "bindings": [
      {
        "tProvider": {
          "type": "uri",
          "value": "http://ndis-network.org/0000004100"
        },
        "PaymentCount": {
          "datatype": "http://www.w3.org/2001/XMLSchema#integer",
          "type": "literal",
          "value": "219"
        },
        "PaymentAmount": {
          "datatype": "http://www.w3.org/2001/XMLSchema#float",
          "type": "literal",
          "value": "1515.7751"
        }
      }
    ]
  }
}


,tProvider,PaymentCount,PaymentAmount
1,http://ndis-network.org/0000004100,219,1515.7751
